In [2]:
from pathlib import Path

root = Path()
children = list(root.iterdir())
course_dirs = list(
    filter(
        lambda x: x.is_dir() and x.name not in [".git", ".DS_Store"],
        children,
    )
)
course_names = list(map(lambda x: x.name, course_dirs))

In [5]:
%pip install 'unstructured'
%pip install 'unstructured[pdf]'

Defaulting to user installation because normal site-packages is not writeable
  Using cached unstructured-0.11.2-py3-none-any.whl.metadata (25 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 16.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.6 MB/s eta 0:00:0000:01
  Using cached emoji-2.9.0-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached python_iso639-2023.6.15-py3-none-any.whl.metadata (13 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached unstructured-0.11.2-py3-none-any.whl (1.7 MB)
Using cached emoji-2.9.0-py2.py3-none-any.whl (397 kB)
Using cached python_iso639-2023.6.15-py3-none-any.whl (275

In [7]:
import unstructured.partition.pdf as pdf

/Users/samarthgoel/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
partitioned_elements = pdf.partition_pdf(
    "test.pdf",
    include_page_breaks=True,
    strategy="hi_res",
    infer_table_structure=True,
    extract_images_in_pdf=True,
    output_type="dict",
    image_output_dir_path="testpdf_images",
)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
partitioned_elements[:5]

In [27]:
partitioned_elements[10].metadata.coordinates

CoordinatesMetadata(points=((181.5944444444445, 818.709444444444), (181.5944444444445, 918.8329077777773), (1540.0108633333334, 918.8329077777773), (1540.0108633333334, 818.709444444444)), system=<unstructured.documents.coordinates.PixelSpace object at 0x2bbe5d4f0>)

In [30]:
for i in range(10, 25):
    print(partitioned_elements[i])
    print()

• For multiple answer questions, ﬁll in the bubbles for ALL correct choices: there may be more than one correct choice, but there is always at least one correct choice. NO partial credit on multiple answer questions: the set of all correct answers must be checked.

First name Last name SID First and last name of student to your left First and last name of student to your right

1



Q1. [60 pts] Multiple Answer

Fill in the bubbles for ALL correct choices: there may be more than one correct choice, but there is always at least one correct choice. NO partial credit: the set of all correct answers must be checked.

1+exp θ ) for some θ ∈ R. What is the MLE estimator of θ? 1 (a) [3 pts] Let X ∼ Bernoulli(

(cid:13) X (cid:13) 0 (cid:13) 1

(cid:13) Does not exist.

(b) [3 pts] Let Y ∼ N(Xθ, In) for some unknown θ ∈ Rd and some known X ∈ Rn×d that has full column rank and d < n. What is the MLE estimator of θ?

(cid:13) Y + Z ∀Z ∈ Null(X)

(cid:13) (X(cid:62)X)−1X(cid:62)Y (cid:13) X(cid:6